# TSA Chapter 7: Quiz 5 - VECM vs VAR in Differences

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch7/TSA_ch7_quiz5_vecm_forecast/TSA_ch7_quiz5_vecm_forecast.ipynb)

Comparing forecasts from VECM and VAR in differences for cointegrated series.

In [ ]:
!pip install numpy matplotlib -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']
plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})
def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
np.random.seed(99)
T_train = 150
T_forecast = 30
T_total = T_train + T_forecast

# Generate cointegrated pair: y_t = beta * x_t + stationary error
beta = 1.2
alpha_y = -0.3  # error correction speed for Y
alpha_x = 0.1   # error correction speed for X

x = np.zeros(T_total)
y = np.zeros(T_total)
x[0] = 10; y[0] = beta * x[0] + 2

for t in range(1, T_total):
    ecm = y[t-1] - beta * x[t-1] - 2  # equilibrium error
    x[t] = x[t-1] + alpha_x * ecm + np.random.normal(0, 0.3)
    y[t] = y[t-1] + alpha_y * ecm + np.random.normal(0, 0.3)

# True values in forecast period
true_y = y[T_train:]
true_x = x[T_train:]

# VECM forecast (uses error correction term) - better for cointegrated data
vecm_y = np.zeros(T_forecast)
vecm_x = np.zeros(T_forecast)
vecm_y[0] = y[T_train-1]; vecm_x[0] = x[T_train-1]
for t in range(1, T_forecast):
    ecm = vecm_y[t-1] - beta * vecm_x[t-1] - 2
    vecm_x[t] = vecm_x[t-1] + alpha_x * ecm + np.random.normal(0, 0.15)
    vecm_y[t] = vecm_y[t-1] + alpha_y * ecm + np.random.normal(0, 0.15)

# VAR in differences forecast (ignores long-run relationship) - drifts apart
var_d_y = np.zeros(T_forecast)
var_d_x = np.zeros(T_forecast)
var_d_y[0] = y[T_train-1]; var_d_x[0] = x[T_train-1]
# Use average differences from training data
mean_dy = np.mean(np.diff(y[:T_train]))
mean_dx = np.mean(np.diff(x[:T_train]))
for t in range(1, T_forecast):
    var_d_x[t] = var_d_x[t-1] + mean_dx + np.random.normal(0, 0.3)
    var_d_y[t] = var_d_y[t-1] + mean_dy + np.random.normal(0, 0.3)

# --- Plot ---
fig, axes = plt.subplots(1, 2, figsize=(10, 4.5))
forecast_idx = np.arange(T_train, T_total)
train_idx = np.arange(T_train)

# VECM forecast
axes[0].plot(train_idx[-50:], y[T_train-50:T_train], color=BLUE, lw=1.0, label='$Y_t$ (actual)')
axes[0].plot(train_idx[-50:], x[T_train-50:T_train], color=RED, lw=1.0, label='$X_t$ (actual)')
axes[0].plot(forecast_idx, vecm_y, color=BLUE, ls='--', lw=1.5, label='$Y_t$ (VECM)')
axes[0].plot(forecast_idx, vecm_x, color=RED, ls='--', lw=1.5, label='$X_t$ (VECM)')
axes[0].axvline(x=T_train, color=GRAY, ls=':', lw=0.8)
axes[0].set_title('VECM Forecast', fontweight='bold')
axes[0].legend(loc='lower right', fontsize=8)
axes[0].set_xlabel('Time')

# Spread: VECM maintains equilibrium
spread_vecm = vecm_y - beta * vecm_x
spread_var = var_d_y - beta * var_d_x

# VAR(Delta) forecast
axes[1].plot(train_idx[-50:], y[T_train-50:T_train], color=BLUE, lw=1.0, label='$Y_t$ (actual)')
axes[1].plot(train_idx[-50:], x[T_train-50:T_train], color=RED, lw=1.0, label='$X_t$ (actual)')
axes[1].plot(forecast_idx, var_d_y, color=BLUE, ls='--', lw=1.5, label='$Y_t$ (VAR$\\Delta$)')
axes[1].plot(forecast_idx, var_d_x, color=RED, ls='--', lw=1.5, label='$X_t$ (VAR$\\Delta$)')
axes[1].axvline(x=T_train, color=GRAY, ls=':', lw=0.8)
axes[1].set_title('VAR in Differences Forecast', fontweight='bold')
axes[1].legend(loc='lower right', fontsize=8)
axes[1].set_xlabel('Time')

plt.tight_layout()
save_chart(fig, 'ch7_quiz5_vecm_forecast')
plt.show()

print('\nQuiz: For cointegrated series, which model better preserves the long-run relationship?')
print('  (a) VECM  (b) VAR in differences  (c) Both equally  (d) Neither')
print('\nHint: VAR in differences discards the error correction term, so forecasts may drift apart.')
print(f'  VECM spread std: {np.std(spread_vecm):.3f}')
print(f'  VAR(Delta) spread std: {np.std(spread_var):.3f}')